# FT Replicate 1

In [1]:
import numpy as np
import torch
import torch.nn as nn
from utils import training, callback, evaluating, attention, data
from sklearn import datasets, model_selection
import skorch
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
#####################################################
# Configuration
#####################################################

dataset = "christine"
aggregator = "cls"

print(f"Using -- Dataset:{dataset} Aggregator:{aggregator}")

Using -- Dataset:christine Aggregator:cls


In [3]:
id = 554 #41142 #this is the ID of the Iris dataset in OpenML  

# Load the Iris dataset
df = data.read_dataset_by_id(id) #this function returns a dictionary with the dataset's data and metadata

INFO:root:Starting [get] request for the URL https://www.openml.org/api/v1/xml/data/554
INFO:root:0.5146148s taken for [get] request for the URL https://www.openml.org/api/v1/xml/data/554
INFO:root:Starting [get] request for the URL https://www.openml.org/api/v1/xml/data/features/554
INFO:root:0.6942627s taken for [get] request for the URL https://www.openml.org/api/v1/xml/data/features/554
INFO:root:Starting [get] request for the URL https://www.openml.org/api/v1/xml/data/qualities/554
INFO:root:0.6998386s taken for [get] request for the URL https://www.openml.org/api/v1/xml/data/qualities/554
INFO:root:Starting [get] request for the URL https://api.openml.org/data/v1/download/52667/mnist_784.arff
INFO:root:7.1824317s taken for [get] request for the URL https://api.openml.org/data/v1/download/52667/mnist_784.arff
INFO:urllib3.poolmanager:Redirecting http://openml1.win.tue.nl/datasets/0000/0554/dataset_554.pq -> https://openml1.win.tue.nl/datasets/0000/0554/dataset_554.pq
INFO:urllib3.

In [4]:
df.keys()

dict_keys(['features', 'outputs', 'target', 'labels', 'columns', 'categorical', 'categories', 'n_categorical', 'numerical', 'n_numerical'])

In [5]:
df["categories"]

{}

In [6]:
print(f"In total there are {len(df['numerical'])} numerical columns and {len(df['categorical'])} categorical columns \n")
print(f"The categorical columns are: {df['categorical']}")
print(f"The numerical columns are: {df['numerical']}")

In total there are 784 numerical columns and 0 categorical columns 

The categorical columns are: []
The numerical columns are: ['pixel1' 'pixel2' 'pixel3' 'pixel4' 'pixel5' 'pixel6' 'pixel7' 'pixel8'
 'pixel9' 'pixel10' 'pixel11' 'pixel12' 'pixel13' 'pixel14' 'pixel15'
 'pixel16' 'pixel17' 'pixel18' 'pixel19' 'pixel20' 'pixel21' 'pixel22'
 'pixel23' 'pixel24' 'pixel25' 'pixel26' 'pixel27' 'pixel28' 'pixel29'
 'pixel30' 'pixel31' 'pixel32' 'pixel33' 'pixel34' 'pixel35' 'pixel36'
 'pixel37' 'pixel38' 'pixel39' 'pixel40' 'pixel41' 'pixel42' 'pixel43'
 'pixel44' 'pixel45' 'pixel46' 'pixel47' 'pixel48' 'pixel49' 'pixel50'
 'pixel51' 'pixel52' 'pixel53' 'pixel54' 'pixel55' 'pixel56' 'pixel57'
 'pixel58' 'pixel59' 'pixel60' 'pixel61' 'pixel62' 'pixel63' 'pixel64'
 'pixel65' 'pixel66' 'pixel67' 'pixel68' 'pixel69' 'pixel70' 'pixel71'
 'pixel72' 'pixel73' 'pixel74' 'pixel75' 'pixel76' 'pixel77' 'pixel78'
 'pixel79' 'pixel80' 'pixel81' 'pixel82' 'pixel83' 'pixel84' 'pixel85'
 'pixel86' 'pixel87

In [7]:
print(type(df))

<class 'dict'>


In [8]:
df_pandas = df["features"]
df_pandas
print(type(df_pandas))

<class 'pandas.core.frame.DataFrame'>


In [9]:
#First numerical and then the categorcial columns

categorical_features = df['categorical'].tolist()
numerical_features = df['numerical'].tolist()
#target = df["target"]

numerical_features = df_pandas[numerical_features]  # Assuming numerical_features is a list of column names
categorical_features = df_pandas[categorical_features]  # Assuming categorical_features is a list of column names
#target = df_pandas[target]  # Assuming target is a column name

#print(target)

print(type(categorical_features))
print(type(numerical_features))
print(type(df["target"]))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'str'>


In [10]:
df_ordered = pd.concat([numerical_features,categorical_features], axis=1)

In [11]:
df_ordered

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Suppose 'df' is your DataFrame and 'categorical_columns' is a list of column names with categorical features
for col in categorical_features:
    df_ordered[col], _ = pd.factorize(df_ordered[col])

In [13]:
df_ordered

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
X = df_ordered.values
y = df["outputs"].codes

print(type(X))
print(type(y))  

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [15]:
"""
Dataset metadata definition.

    n_instances: Number of instances (rows) in your dataset.
    n_numerical: Number of numerical features in your dataset.
    n_categorical: List of the number of categories for each categorical column.
    n_labels: Number of classification labels.
    
"""

n_instances = len(X)
n_numerical = df["n_numerical"]
n_categorical = df["n_categorical"]
n_labels = len(df["labels"].keys())

In [16]:
n_instances

70000

In [17]:
#####################################################
    # Split data
#####################################################

from sklearn import datasets, model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1)
train_indices, val_indices = model_selection.train_test_split(np.arange(X_train.shape[0]), test_size=1/9) #1/9 of train is equal to 10% of total

In [18]:
"""
Transformer hyperparameters definition.

n_heads: Number of heads oneach Transformer Encoder.
embed_dim: The embeddings' dimension.
n_layers: Number of stacked Transformer Encoders.
ff_pw_size: Position-wise Feed Forward network hidden layer size.
attn_dropout: Dropout applied in the Multi-head self-attention mechanism.
ff_dropout: Position-wise Feed Forward network dropout.
aggregator: Aggregator to use. Must be in {concatenate, cls, max, mean, sum, rnn}
aggregator_parameters: If the aggregator is different from rnn, set to None. Otherwise, a dictionary expecting:
                        cell: Cellused in the RNN. Must be one of {GRU, LSTM}
                        output_size: Recurrent neural network hidden size 
                        num_layers: Number of stacked layers in the RNN
                        dropout: Dropout applied to the RNN
    }
decoder_hidden_units: List of hidden layer's sizes of the decoder MLP.
decoder_activation_fn: Activation function used in the hidden layers
need_weights: Set True if you require the attention cubes. During training is recommended to set it to False.
numerical_passthrough: False if numerical features will be processed by the Multi-head self-attention mechanism.
"""


n_heads = 4 # In average 4 works better
embed_dim = 4 # In average 256 works better
n_layers = 3
ff_pw_size = 30  #this value because of the paper 
attn_dropout = 0.3 #paper
ff_dropout = 0.1 #paper value
aggregator = "cls"
aggregator_parameters = None
decoder_hidden_units = [128,64] #paper value
decoder_activation_fn = nn.ReLU()
need_weights = False
numerical_passthrough = False

module = training.build_module(
    n_categorical, # List of number of categories
    n_numerical, # Number of numerical features
    n_heads, # Number of heads per layer
    ff_pw_size, # Size of the MLP inside each transformer encoder layer
    n_layers, # Number of transformer encoder layers    
    n_labels, # Number of output neurons
    embed_dim,
    attn_dropout, 
    ff_dropout, 
    aggregator, # The aggregator for output vectors before decoder
    rnn_aggregator_parameters=aggregator_parameters,
    decoder_hidden_units=decoder_hidden_units,
    decoder_activation_fn=decoder_activation_fn,
    need_weights=need_weights,
    numerical_passthrough=numerical_passthrough
)

print(module)


model = skorch.NeuralNetClassifier(
            module = module,
            criterion=torch.nn.CrossEntropyLoss,
            optimizer=torch.optim.AdamW,
            device= "cuda", #cuda" if torch.cuda.is_available() else
            batch_size=32,
            max_epochs=100,
            train_split=skorch.dataset.ValidSplit(((train_indices, val_indices),)),
            callbacks=[
                ("balanced_accuracy", skorch.callbacks.EpochScoring("balanced_accuracy", lower_is_better=False)),
                ("accuracy", skorch.callbacks.EpochScoring("accuracy", lower_is_better=False)),
                ("duration", skorch.callbacks.EpochTimer())
            ],
            optimizer__lr=1e-4,
            optimizer__weight_decay=1e-4
        )
    

TabularTransformer(
  (numerical_encoder): NumericalEncoder()
  (transformer_encoder): TTransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TTransformerEncoderLayer(
        (pre_norm_1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        (pre_norm_2): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        (self_attn): MultiheadAttentionContainer(
          (in_proj_container): InProjContainer(
            (query_proj): Linear(in_features=4, out_features=4, bias=True)
            (key_proj): Linear(in_features=4, out_features=4, bias=True)
            (value_proj): Linear(in_features=4, out_features=4, bias=True)
          )
          (attention_layer): ScaledDotProduct()
          (out_proj): Linear(in_features=4, out_features=4, bias=True)
        )
        (ff_network): Sequential(
          (0): Linear(in_features=4, out_features=30, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=30, out_featu

/home/diego/anaconda3/envs/tabtrans/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [19]:
"""
Training and validation
"""

model = model.fit(X={
        "x_numerical": X_train[:, :n_numerical].astype(np.float32),
        "x_categorical": X_train[:, n_numerical:].astype(np.int32)
        }, 
        y=y_train.astype(np.int64)
    )

  epoch    accuracy    balanced_accuracy    train_loss    valid_acc    valid_loss       dur
-------  ----------  -------------------  ------------  -----------  ------------  --------
      1      0.1371               0.1329        2.3003       0.1371        2.2935  252.7765
      2      0.3161               0.3168        1.9269       0.3161        1.7194  253.0228
      3      0.3399               0.3390        1.7051       0.3399        1.6705  246.7527
      4      0.3540               0.3529        1.6616       0.3540        1.6379  246.7895
      5      0.3684               0.3671        1.6297       0.3684        1.6080  246.7533
      6      0.3856               0.3841        1.6019       0.3856        1.5781  246.7327
      7      0.3900               0.3876        1.5725       0.3900        1.5424  246.8606
      8      0.4116               0.4090        1.5358       0.4116        1.4945  253.3012
      9      0.4314               0.4269        1.4924       0.4314        1.440